In [1]:
  from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


- 데이터를 읽어온다.
- 셔플을 한다.(seed=777)
- 1000개 -> training, 385 -> test
- label = Baseline Histological Grading, Baseline Histological
- 나머지는 feature
- ACC, PREC, SENS, F1, AUROC 

In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.utils import shuffle
import torch
from torch import nn
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

- batch에서 normalization을 하도록 한다.
- 데이터 preprocssing을 먼저 해주고 

In [0]:
# 다음 code skeleton을 사용하여 DataLoader를 구성할 수 있습니다.
# (TODO에 해당하는 내용을 채우기 전에는 실행되지 않습니다)

# You should build your custom dataset as below
class CustomDataset_27(torch.utils.data.Dataset):

# 우리가 class 생성시에 해줄것은
# self.len 에 길이를 넣어주고
# self.x_data에 x 데이터를
# self.y_data에 y 데이터를 넣어주고
# 인덱스에 따라 i 번째 데이터를 잘 리턴해주면 됩니다.

  def __init__(self,test):
    # TODO
    # 1. initialize file paths or a list of file names
    # 데이터셋의 전처리를 해주는 부분

    # data = pd.read_csv('drive/My Drive/2020winter-MLcamp/content/HCV-Egy-Data.csv')
    data = pd.read_csv('drive/My Drive/HCV-Egy-Data.csv')
    data = shuffle(data,random_state=777)
    data = data.to_numpy()
    # data = torch.from_numpy(data.astype('float'))

    # print(data)
    if test==False:
      data=data[:1000]
    else:
      data=data[1000:]
    x_data=data[:,:27]
    self.x_data = torch.from_numpy(x_data.astype('float'))
    y_data = data[:,27]
    #y_data=np.eye(16)[y_data - 1]
    y_data = y_data - 1
    self.y_data = torch.from_numpy(y_data.astype('int64'))
    self.length = len(data)

  def __getitem__(self, index):
    return self.x_data[index],self.y_data[index]

  def __len__(self):
    return self.length


class CustomDataset_28(torch.utils.data.Dataset):

# 우리가 class 생성시에 해줄것은
# self.len 에 길이를 넣어주고
# self.x_data에 x 데이터를
# self.y_data에 y 데이터를 넣어주고
# 인덱스에 따라 i 번째 데이터를 잘 리턴해주면 됩니다.

  def __init__(self,test):
    # TODO
    # 1. initialize file paths or a list of file names
    # 데이터셋의 전처리를 해주는 부분

    # data = pd.read_csv('drive/My Drive/2020winter-MLcamp/content/HCV-Egy-Data.csv')
    data = pd.read_csv('drive/My Drive/HCV-Egy-Data.csv')
    data = shuffle(data,random_state=777)
    data=data.to_numpy()
    # data = torch.from_numpy(data)

    # print(data)
    if test==False:
      data=data[:1000]
    else:
      data=data[1000:]
    
    x_data=data[:,:27]
    self.x_data = torch.from_numpy(x_data.astype('float'))
    y_data = data[:,28]
    #y_data=np.eye(16)[y_data - 1]
    y_data = y_data - 1
    self.y_data = torch.from_numpy(y_data.astype('int64'))
    self.length = len(data)

  def __getitem__(self, index):
    return self.x_data[index],self.y_data[index]

  def __len__(self):
    return self.length



# You can then use the prebuilt data loader
train_dataset_27 = CustomDataset_27(test=False)
test_dataset_27= CustomDataset_27(test=True)
train_dataset_28 = CustomDataset_28(test=False)
test_dataset_28= CustomDataset_28(test=True)


In [0]:
input_size = 27
num_classes_27 = 16
num_classes_28 = 2
batch_size = 100
# hidden_size_1 = 256
# hidden_size_2 = 128
learning_rate = 0.001

train_loader_27 = torch.utils.data.DataLoader(dataset = train_dataset_27,
                                           batch_size = batch_size,
                                           shuffle = True)

test_loader_27 = torch.utils.data.DataLoader(dataset = test_dataset_27,
                                          batch_size = batch_size,
                                          shuffle = False)
train_loader_28 = torch.utils.data.DataLoader(dataset = train_dataset_28,
                                           batch_size = batch_size,
                                           shuffle = True)
test_loader_28 = torch.utils.data.DataLoader(dataset = test_dataset_28,
                                           batch_size = batch_size,
                                           shuffle = True)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Fully connected neural network with one hidden layer

256, 128, 
class FFNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(FFNet, self).__init__()
        self.net = nn.Sequential(
          nn.Linear(input_size, 256),
          nn.BatchNorm1d(256), # 이게 바로 batchnormalization을 해주는 것으로 이미지 데이터 같은 경우는 nn.BatchNorm2d를 사용합니다.
          nn.ReLU(),
        
          nn.Linear(256, 128),
          nn.BatchNorm1d(128),
          nn.ReLU(),
        
          nn.Linear(128, 64),
          nn.BatchNorm1d(64),
          nn.ReLU(),
        
          nn.Linear(64, num_classes)
        ) # sequential은 안에 있는 내용물들에 순차적으로 대입해주는 겁니다. 아래 보시면 forward가 한 줄에 output이 나오죠?
    def forward(self, x):
        out = self.net(x)
        return out


def train_ffnet(model, train_loader, num_epochs):
  model.train()
  total_step = len(train_loader)
  for epoch in range(num_epochs):
    for i, (features, labels) in enumerate(train_loader):
      optimizer.zero_grad()
      
      features = features.float().to(device)
      labels = labels.long().to(device)
      outputs = model(features)
      loss = criterion(outputs, labels)

      loss.backward()
      optimizer.step()

      if(i+1) % 300 == 0:
        print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))
        
def test_ffnet(model, test_loader):
  model.eval()
  with torch.no_grad():
    correct=0
    total =0
    for features, labels in test_loader:
      features = features.float().to(device)
      labels = labels.float().to(device)
      outputs = model(features)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == (labels)).sum().item()
    print('Accuracy of the network on the 1000 test inputs: {}%'.format(100*correct/total))


In [5]:
# declare a model object  (instantiation)
model_27 = FFNet(input_size, num_classes_27).to(device)
model_28 = FFNet(input_size, num_classes_28).to(device)
# Set the loss and optimizer

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_27.parameters(), lr = learning_rate)

train_ffnet(model_27, train_loader_27, num_epochs = 200)
test_ffnet(model_27, test_loader_27)
train_ffnet(model_28, train_loader_28, num_epochs = 300)
test_ffnet(model_28, test_loader_28)

NameError: ignored

- multi label classification (MLC):
X에 대한 다수의 y들이 있는데,
이 y들의 관계를 모델에 반영하자는 것이 MLC

- Multi Instance Muli Label (MIML) :
x가 여러개, y가 3개가 있다면

x한줄 - y1
x한줄 - y2
x한줄 - y3
instance의 3배의 갯수의 인스턴스가 만들어짐




- batch size = 100
- 512,512,256,128 모델
- epoch = 300
- Accuracy of the network on the 1000 test inputs: 5.714285714285714%
- Accuracy of the network on the 1000 test inputs: 27.532467532467532%

-
-
-

- batch size = 100
- 512,512,256,128 모델
- epoch = 200, 300
- Accuracy of the network on the 1000 test inputs: 9.35064935064935%
- Accuracy of the network on the 1000 test inputs: 27.532467532467532%

-
-
-


- batch size = 100
- 512,256,128 모델
- epoch = 200, 300
- Accuracy of the network on the 1000 test inputs: 9.35064935064935%
- Accuracy of the network on the 1000 test inputs: 25.714285714285715%

-
-
-

- batch size = 100
- 256,128 모델
- epoch = 200, 300
- Accuracy of the network on the 1000 test inputs: 9.35064935064935%
- Accuracy of the network on the 1000 test inputs: 29.09090909090909%









